## Load Packages

In [66]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
from collections import deque

## Functions

### Loading in Image

In [67]:
def load_image(annotation_path, slice_path, plots = True):
    # Reading in images and converting to RGB for plotting
    annotation = cv.cvtColor(cv.imread(annotation_path), cv.COLOR_BGR2RGB)
    slice = cv.cvtColor(cv.imread(slice_path), cv.COLOR_BGR2RGB)

    # Creating plots
    if plots is True:
        fig, axes = plt.subplots(1, 2, figsize = (8, 8))
        plt.subplots_adjust(wspace = 0.3)
        axes[0].imshow(annotation)
        axes[0].set_title(os.path.basename(annotation_path))
        axes[1].imshow(slice)
        axes[1].set_title(os.path.basename(slice_path))
        plt.show()

    # Returning array sizes to verify that they are the same
    print(f"Annotation Shape: {annotation.shape} \nSlice Shape: {slice.shape}")

    # Return annotation and slice arrays
    return annotation, slice

### Getting Masks for Each Epithelium "Object"

In [68]:
def get_masks(annotation, slice, plots = True):
    # Convert annotated image to grayscale
    annotation_gray = cv.cvtColor(annotation, cv.COLOR_RGB2GRAY)
    # All pixels with a grayscale value above 127 converted to white, all below converted to black
    thresh = cv.threshold(annotation_gray, 127, 255, 0)[1]
    # Get contours
    contours, hierarchy = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)
    # Create list to store individual object masks and coordinates
    masks = []
    # Get object masks
    for count, i in enumerate(np.argwhere(hierarchy[0, :, 3] == 0).ravel()):
        mask_dict = {}
        mask_dict['mask'] = np.zeros_like(annotation_gray)
        cv.drawContours(mask_dict['mask'], contours, i, 255, -1)
        mask_dict['epithelium_points'] = np.transpose(np.nonzero(mask_dict['mask']))
        masks.append(mask_dict)
    
    # Plot object masks and original slice
    if plots is True:
        number = len(masks)
        fig, axes = plt.subplots(1, number + 1, figsize = (4 * (number + 1), 4 * (number + 1)))
        plt.subplots_adjust(wspace = 0.3)
        for i in range(number):
            mask = masks[i]['mask']
            axes[i].imshow(mask, 'gray', vmin = 0, vmax = 255)
            axes[i].set_title(f'Mask {i + 1}')
        axes[number].imshow(slice)
        axes[number].set_title("Original")
        plt.show()

    # Return list of masks
    return masks

### Centering

In [69]:
def center(mask, current_point, number_centering_moves = 3):
    # Center along shorter "width" number_centering_moves times
    for i in range(number_centering_moves):
        # Finds first pixel that's non-epithelium (0, 80, 120) or patched (160, 210) in all four directions from current point
        # And takes one step toward the point to get back on the epithelium
        horizontal_zero_indices = np.argwhere(mask[current_point[0], :] <= 210)
        left_bound = np.where(horizontal_zero_indices <= current_point[1], horizontal_zero_indices, -np.inf).max() + 1
        right_bound = np.where(horizontal_zero_indices > current_point[1], horizontal_zero_indices, np.inf).min() - 1
        vertical_zero_indices = np.argwhere(mask[:, current_point[1]] <= 210)
        top_bound = np.where(vertical_zero_indices <= current_point[0], vertical_zero_indices, -np.inf).max() + 1
        bottom_bound = np.where(vertical_zero_indices > current_point[0], vertical_zero_indices, np.inf).min() - 1
        
        # Check if on edge
        on_horizontal_edge = False
        on_vertical_edge = False
        if right_bound == current_point[1] or left_bound == current_point[1]:
            on_horizontal_edge = True
        if bottom_bound == current_point[0] or top_bound == current_point[0]:
            on_vertical_edge = True
        
        # Calculate widths
        widths = np.array([right_bound - left_bound, bottom_bound - top_bound])

        # Move to center of shorter width if not on an edge
        shorter_width = np.argmin(widths)
        if not on_horizontal_edge and not on_vertical_edge:
            if shorter_width == 0:
                current_point[1] = (right_bound + left_bound) // 2
            else:
                current_point[0] = (bottom_bound + top_bound) // 2
            continue
        
        # If on an edge, move to center of width that isn't the edge
        # If on both edges, just defaults to moving horizontally
        # Moving in both directions simultaneously without computational expense of recalculating after first move is dangerous
        # Could end up outside epithelium
        if on_horizontal_edge:
            current_point[0] = (bottom_bound + top_bound) // 2
        else:
            current_point[1] = (right_bound + left_bound) // 2
        
    directions = ['right', 'left'] if shorter_width == 1 else ['up', 'down']

    # Return point to draw squares off
    # whether to go vertically/horizontally (shorter_width == 0 or 1) 
    # and the width in that direction
    return current_point, directions, np.min(widths)

### Get Initial Corners

In [70]:
def get_corners(point, width, direction):
    corners = np.array([0, 0, 0, 0])
    half_width = width // 2
    # If width is odd amount, add back 1 in certain case to ensure width reaches both edges
    floor_division_adjustment = 0 if width % 2 == 0 else 1
    x, y = point[1], point[0]
    
    # Direction square will be drawn determines calculation of "corners" (really x and y values) based on current point
    if direction == 'right':
        corners[0] = x + width
        corners[1] = y + half_width + floor_division_adjustment
        corners[2] = x
        corners[3] = y - half_width
    elif direction == 'down':
        corners[0] = x + half_width + floor_division_adjustment
        corners[1] = y + width
        corners[2] = x - half_width
        corners[3] = y
    elif direction == 'left':
        corners[0] = x
        corners[1] = y + half_width + floor_division_adjustment
        corners[2] = x - width
        corners[3] = y - half_width
    else:
        corners[0] = x + half_width + floor_division_adjustment
        corners[1] = y
        corners[2] = x - half_width
        corners[3] = y - width
        
    return corners

### Check Square Validity

In [71]:
def get_corner_validity(corners, mask):
    # Getting indexes for axes that determine square
    right = corners[0]
    bottom = corners[1]
    left = corners[2]
    top = corners[3]

    # Checking corner validity
    # Basically the condition here is at least one pixel adjacent to each corner or the corner itself 
    # has to fall on a non-epithelial pixel (corresponds to values of 0, 80 and 120)
    upper_right, lower_right, lower_left, upper_left = True, True, True, True
    if top > 1 and right < (mask.shape[1] - 2):
        if mask[top, right + 1] > 120 and mask[top - 1, right] > 120 and mask[top, right] > 120:
            upper_right = False
    if bottom < (mask.shape[0] - 2) and right < (mask.shape[1] - 2):
        if mask[bottom, right + 1] > 120 and mask[bottom + 1, right] > 120 and mask[bottom, right] > 120:
            lower_right = False
    if bottom < (mask.shape[0] - 2) and left > 1:
        if mask[bottom, left - 1] > 120 and mask[bottom + 1, left] > 120 and mask[bottom, left] > 120:
            lower_left = False
    if top > 1 and left > 1:
        if mask[top, left - 1] > 120 and mask[top - 1, left] > 120 and mask[top, left] > 120:
            upper_left = False
    corner_validity_array = np.array([upper_right, lower_right, lower_left, upper_left])
    corner_validity = True if False not in corner_validity_array else False
    
    return corner_validity, corner_validity_array

In [72]:
def get_side_validity(corners, mask):
    # Getting indexes for axes that determine square
    right = corners[0]
    bottom = corners[1]
    left = corners[2]
    top = corners[3]

    # Checking side validity
    # Looking at square of pixels directly around the patch
    # A maximum of two sides in that square can have epithelium pixels on them
    outer_right = int(min(right + 1, (mask.shape[1] - 1)))
    outer_bottom = int(min(bottom + 1, (mask.shape[0] - 1)))
    outer_left = int(max(left - 1, 0))
    outer_top = int(max(top - 1, 0))
    
    right_valid, bottom_valid, left_valid, top_valid = True, True, True, True
    if np.max(mask[outer_top:outer_bottom, outer_right]) > 120:
        right_valid = False
    if np.max(mask[outer_bottom, outer_left:outer_right]) > 120:
        bottom_valid = False
    if np.max(mask[outer_top:outer_bottom, outer_left]) > 120:
        left_valid = False
    if np.max(mask[outer_top, outer_left:outer_right]) > 120:
        top_valid = False
    side_validity_array = np.array([right_valid, bottom_valid, left_valid, top_valid])
    side_validity = np.sum(side_validity_array)

    return side_validity, side_validity_array

In [73]:
def get_overlap(corners, mask):
    # Getting indexes for axes that determine square
    right = int(min(corners[0], (mask.shape[1] - 1)))
    bottom = int(min(corners[1], (mask.shape[0] - 1)))
    left = int(max(corners[2], 0))
    top = int(max(corners[3], 0))

    # Checking overlap
    # Looking at sides, if there are any values of 160
    # This is faster than checking the entire square and equally accurate, even if it is more verbose for what it's worth
    # If we hypothetically decide to allow overlap, then just check % of 160s in potential square and see if under threshold
    right_overlap, bottom_overlap, left_overlap, top_overlap = False, False, False, False
    if 120 in mask[top:bottom, right] or 160 in mask[top:bottom, right]:
        right_overlap = True
    if 120 in mask[top:bottom, right] or 160 in mask[bottom, left:right]:
        bottom_overlap = True
    if 120 in mask[top:bottom, right] or 160 in mask[top:bottom, left]:
        left_overlap = True
    if 120 in mask[top:bottom, right] or 160 in mask[top, left:right]:
        top_overlap = True
    overlap_array = np.array([right_overlap, bottom_overlap, left_overlap, top_overlap])

    # If there is overlap, overlap = True and under these conditions, square will just be discarded
    # As stated in comments above, setting a maximum overlap threshold is possible
    overlap = True if True in overlap_array else False

    return overlap, overlap_array

### Fixing Validity

In [74]:
def fix_corner_validity(corners, mask, direction, corner_validity):
    # If no corners have issues, don't do anything
    if corner_validity != True:
        right = corners[0]
        bottom = corners[1]
        left = corners[2]
        top = corners[3]

        # Fixing Invalid Corners (only check two that weren't on original width)
        # Find distances to edges for the invalid corners, move those corners to the edges
        # Then add both distances axis connecting those corners to keep shape of square
        if direction == 'right':
            vertical_zero_indices = np.argwhere(mask[:, right] <= 120)
            distance_down = np.where(vertical_zero_indices > bottom, vertical_zero_indices, np.inf).min() - 1 - bottom
            distance_up = top - np.where(vertical_zero_indices < top, vertical_zero_indices, -np.inf).max() + 1
            corners = np.array([right + distance_down + distance_up, bottom + distance_down, left, top - distance_up])
        elif direction == 'down':
            horizontal_zero_indices = np.argwhere(mask[bottom, :] <= 120)
            distance_right = np.where(horizontal_zero_indices > right, horizontal_zero_indices, np.inf).min() - 1 - right
            distance_left = left - np.where(horizontal_zero_indices < left, horizontal_zero_indices, -np.inf).max() + 1
            corners = np.array([right + distance_right, bottom + distance_right + distance_left, left - distance_left, top])
        elif direction == 'left':
            vertical_zero_indices = np.argwhere(mask[:, left] <= 120)
            distance_down = np.where(vertical_zero_indices > bottom, vertical_zero_indices, np.inf).min() - 1 - bottom
            distance_up = top - np.where(vertical_zero_indices < top, vertical_zero_indices, -np.inf).max() + 1
            corners = np.array([right, bottom + distance_down, left - distance_down - distance_up, top - distance_up])
        else:
            horizontal_zero_indices = np.argwhere(mask[top, :] <= 120)
            distance_right = np.where(horizontal_zero_indices > right, horizontal_zero_indices, np.inf).min() - 1 - right
            distance_left = left - np.where(horizontal_zero_indices < left, horizontal_zero_indices, -np.inf).max() + 1
            corners = np.array([right + distance_right, bottom, left - distance_left, top - distance_right - distance_left])
    
    return corners

In [75]:
def fix_side_validity(corners, mask, direction, side_validity):
    # If there are two or more valid sides, don't do anything
    if side_validity < 2:
        right = int(min(corners[0], (mask.shape[1] - 1)))
        bottom = int(min(corners[1], (mask.shape[0] - 1)))
        left = int(max(corners[2], 0))
        top = int(max(corners[3], 0))

        # Fixing invalid sides
        # Process here will be to check the three sides (excluding the one the original width lies on)
        # And find the distances from the side to the furthest epithelium point outside
        # Whichever two sides have the shortest distances will expand by that amount
        # At that point, the square should be valid because the corners have already been verified and now the entire width
        # of the epithelium in that region is inside the square
        right_distance = np.min(np.where(np.all(mask[top:bottom, right:] < 160, axis = 0))[0])
        bottom_distance = np.min(np.where(np.all(mask[bottom:, left:right] < 160, axis = 1))[0])
        left_distance = np.max(np.where(np.all(mask[top:bottom, :left] < 160, axis = 0))[0]) - left
        top_distance = np.max(np.where(np.all(mask[:top, left:right] < 160, axis = 1))[0]) - top

        if direction == 'right':
            left_distance = 0
        elif direction == 'down':
            top_distance = 0
        elif direction == 'left':
            right_distance = 0
        else:
            bottom_distance = 0

        distances = np.array([right_distance, bottom_distance, left_distance, top_distance])
        # If absolute maximum distance only occurs once
        if np.unique(np.abs(distances), return_counts = True)[1][-1] == 1:
            # Set maximum to 0 so adding distances array to corners array doesn't change that axes
            distances = np.where(np.abs(distances) == np.max(np.abs(distances)), 0, distances)
        # Otherwise, if the absolute maximum distance occurs multiple times
        updated_corners = corners + distances

        # Have to maintain square shape, so difference in vertical expansion and horizontal expansion is added to the
        # side that expanded less and was not the original line
        vertical_expansion = distances[1] - distances[3]
        horizontal_expansion = distances[0] - distances[2]
        vertical_horizontal_difference = vertical_expansion - horizontal_expansion
        
        if vertical_horizontal_difference > 0:
            if direction == 'left':
                updated_corners[2] = updated_corners[2] - vertical_horizontal_difference
            else:
                updated_corners[0] = updated_corners[0] + vertical_horizontal_difference
        else:
            if direction == 'down':
                updated_corners[1] = updated_corners[1] - vertical_horizontal_difference
            else:
                updated_corners[3] = updated_corners[3] + vertical_horizontal_difference
    else:
        updated_corners = corners
    
    return updated_corners

### Draw Square

In [76]:
def draw_square(current_point, width, mask, direction, corners_dict):
    # combining all of the helper functions above into one more or less
    corners = get_corners(current_point, width, direction)
    # Check overlap immediately as to not waste time on expanding a doomed square
    if get_overlap(corners, mask)[0] == False:
        corner_validity, _ = get_corner_validity(corners, mask)
        corners = fix_corner_validity(corners, mask, direction, corner_validity)
        side_validity, _ = get_side_validity(corners, mask)
        # Keep adjusting sides until they meet square validity
        while side_validity < 2:
            corners = fix_side_validity(corners, mask, direction, side_validity)
            side_validity, _  = get_side_validity(corners, mask)
        width = corners[0] - corners[2]
        if get_overlap(corners, mask)[0] == False:
            corners_dict[(corners[1], corners[0])] = np.append(corners, width)
    else:
        corners = None
    return corners, corners_dict

### Color Patched Points

In [77]:
def color_points(corners, mask):
    right = int(min(corners[0], (mask.shape[1] - 1)))
    bottom = int(min(corners[1], (mask.shape[0] - 1)))
    left = int(max(corners[2], 0))
    top = int(max(corners[3], 0))

    # Changing color-grading of epithelial and non-epithelial points to show they've been covered
    # 210 for overlapping epithelium patched points
    mask[top:bottom, left:right] = np.where(mask[top:bottom, left:right] == 160, 210, mask[top:bottom, left:right])
    # 160 for patched epithelium
    mask[top:bottom, left:right] = np.where(mask[top:bottom, left:right] == 255, 160, mask[top:bottom, left:right])
    # 120 for overlapping non-epithelium patched points
    mask[top:bottom, left:right] = np.where(mask[top:bottom, left:right] == 80, 120, mask[top:bottom, left:right])
    # 80 for patched non-epithelium
    mask[top:bottom, left:right] = np.where(mask[top:bottom, left:right] == 0, 80, mask[top:bottom, left:right])

    return mask
    

### Get Next Points

In [78]:
def get_next_points(corners, mask):
    # Getting side validities for each side of the square
    _, side_validity_array = get_side_validity(corners, mask)

    right = int(min(corners[0], (mask.shape[1] - 1)))
    bottom = int(min(corners[1], (mask.shape[0] - 1)))
    left = int(max(corners[2], 0))
    top = int(max(corners[3], 0))
    
    next_points = []
    # Using previous square to determine where we can go next
    if side_validity_array[0] == False:
        next_points.append([np.array([np.min(np.where(mask[top:bottom, right])) + top, right]), 'right'])
        next_points.append([np.array([np.max(np.where(mask[top:bottom, right])) + top, right]), 'right'])
    if side_validity_array[1] == False:
        next_points.append([np.array([bottom, np.min(np.where(mask[bottom, left:right])) + left]), 'down'])
        next_points.append([np.array([bottom, np.max(np.where(mask[bottom, left:right])) + left]), 'down'])
    if side_validity_array[2] == False:
        next_points.append([np.array([np.min(np.where(mask[top:bottom, left - 1])) + top, left - 1]), 'left'])
        next_points.append([np.array([np.max(np.where(mask[top:bottom, left - 1])) + top, left - 1]), 'left'])
    if side_validity_array[3] == False:
        next_points.append([np.array([top - 1, np.min(np.where(mask[top - 1, left:right])) + left]), 'up'])
        next_points.append([np.array([top - 1, np.max(np.where(mask[top - 1, left:right])) + left]), 'up'])

    return next_points
        

### Plot Patching on Grayscale Mask

In [79]:
def plot_patches_mask(annotation, corners_dict, plots = True):
    # Function to plot all of the patches on the grayscale mask
    # This will be used for calculating metrics
    annotation_gray = cv.cvtColor(annotation, cv.COLOR_RGB2GRAY)
    annotation_gray = np.where(annotation_gray == 255, 0, 255)
    for corners in corners_dict.values():
        annotation_gray = color_points(corners, annotation_gray)
    if plots == True:
        plt.figure(figsize = (8, 8))
        plt.imshow(annotation_gray, cmap = 'gray', vmin = 0, vmax = 255)
        plt.title("Patched Grayscale Mask")
        plt.show()
    return annotation_gray


### Get Metrics

In [80]:
def patching_metrics(annotation_gray, corners_dict):
    # Epithelium coverage
    patched_epithelial_pixels = np.count_nonzero(annotation_gray == 160)
    patched_overlapping_epithelial_pixels = np.count_nonzero(annotation_gray == 210)
    total_patched_epithelial_pixels = patched_epithelial_pixels + patched_overlapping_epithelial_pixels
    total_epithelial_pixels = np.count_nonzero(annotation_gray == 255) + total_patched_epithelial_pixels
    epithelium_coverage = (total_patched_epithelial_pixels/total_epithelial_pixels) * 100

    # % of patch area outside epithelium
    patched_non_epithelial_pixels = np.count_nonzero(annotation_gray == 80)
    patched_overlapping_non_epithelial_pixels = np.count_nonzero(annotation_gray == 120)
    total_patched_non_epithelial_pixels = patched_non_epithelial_pixels + patched_overlapping_non_epithelial_pixels
    total_patched_pixels = total_patched_epithelial_pixels + total_patched_non_epithelial_pixels
    percentage_patch_area_outside_epithelium = (total_patched_non_epithelial_pixels/total_patched_pixels) * 100

    # Number of patches
    number_of_patches = len(corners_dict)

    # % patch area overlap
    total_overlapping_patched_pixels = patched_overlapping_epithelial_pixels + patched_overlapping_non_epithelial_pixels
    total_patched_pixels = total_overlapping_patched_pixels + patched_epithelial_pixels + patched_non_epithelial_pixels
    percentage_patch_area_overlap = (total_overlapping_patched_pixels/total_patched_pixels) * 100

    print(f"Percentage of Epithelium Covered: {epithelium_coverage:.2f}%")
    print(f"Percentage of Patch Area Outside Epithelium: {percentage_patch_area_outside_epithelium:.2f}%")
    print(f"Number of Patches: {number_of_patches}")
    print(f"Percentage of Patch Area Overlap: {percentage_patch_area_overlap:.2f}%")

### Plot Patches on Original Slice

In [81]:
def plot_patches_slice(slice, corners_dict, plots = True):
    for corners in corners_dict.values():
        right = int(min(corners[0], (slice.shape[1] - 1)))
        bottom = int(min(corners[1], (slice.shape[0] - 1)))
        left = int(max(corners[2], 0))
        top = int(max(corners[3], 0))
        cv.rectangle(slice, (left, top), (right, bottom), (255, 0, 0), 5)
    if plots == True:
        plt.figure(figsize=(8, 8))
        plt.title("Patched Slice")
        plt.imshow(slice)
        plt.show()

### Parent Function

In [82]:
def patch_slice(annotation_path, slice_path, number_centering_moves = 3, plots = True, starting_point = None):
    annotation, slice = load_image(annotation_path, slice_path, plots)
    masks = get_masks(annotation, slice, plots)
    corners_dict = {}
    for entry in masks:
        mask = entry['mask']
        epithelium_points = entry['epithelium_points']
        np.random.seed(1)
        if not starting_point:
            starting_point = epithelium_points[np.random.randint(len(epithelium_points), size = 1)][0]
        current_point, directions, width = center(mask, starting_point, number_centering_moves)
        corners, corners_dict = draw_square(current_point, width, mask, directions[0], corners_dict)
        mask = color_points(corners, mask)
        next_points = deque()
        next_points.extend(get_next_points(corners, mask))
        while next_points:
            point_array = next_points.popleft()
            current_point, potential_direction = point_array[0], point_array[1]
            current_point, directions, width = center(mask, current_point, number_centering_moves)
            if potential_direction in directions:
                corners, corners_dict = draw_square(current_point, width, mask, potential_direction, corners_dict)
            else:
                corners, corners_dict = draw_square(current_point, width, mask, directions[0], corners_dict)
            if corners is not None:
                mask = color_points(corners, mask)
                next_points.extend(get_next_points(corners, mask))
    annotation_gray = plot_patches_mask(annotation, corners_dict, plots)
    patching_metrics(annotation_gray, corners_dict)
    plot_patches_slice(slice, corners_dict, plots)

## Algorithm

In [84]:
# Fill in paths for annotated slice and original .tif image here
annotation_path = "./images/epithelium_extraction/export/h2114153  h&e_4-labels.png"
slice_path = "./images/tif_stains/h2114153  h&e_4.tif"

patch_slice(annotation_path, slice_path)

Annotation Shape: (5832, 2428, 3) 
Slice Shape: (5832, 2428, 3)


KeyboardInterrupt: 